In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt
data = open('data.txt').read()

--2021-01-14 10:54:21--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2021-01-14 10:54:21 (114 MB/s) - ‘data.txt’ saved [93578/93578]



In [2]:
corpus = data.lower().split("\n")

## Yêu cầu: Sinh từ tiếp khi gõ bất cứ một câu nào có chiều dài bất kỳ.

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1

train_sequences = []
train_labels = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        train_sequences.append(token_list[:i])
        train_labels.append(token_list[i])

### 1. Xử lý dữ liệu. Chia features, label

In [6]:
max_sequence_len = max([len(line) for line in train_sequences])
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_len, padding='pre')
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=total_words)

### 2. Xây dựng model
#### Yêu cầu độ chính xác: 80%

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dropout, LSTM, Dense

In [47]:
# Model
model = Sequential()

# Một lớp Embedding
model.add(Embedding(total_words, 100, input_length=max_sequence_len))

# Một lớp Bidirectional với cell là LSTM
model.add(Bidirectional(LSTM(150, return_sequences=True)))

# Một lớp Dropout
model.add(Dropout(0.2))

# Một lớp LSTM
model.add(LSTM(100))

# 2 lớp Dense nhận diện từ tiếp theo
model.add(Dense(total_words//2, activation='relu', kernel_regularizer='l2'))
model.add(Dense(total_words, activation='softmax'))

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [48]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 10, 300)           301200    
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 100)               160400    
_________________________________________________________________
dense_14 (Dense)             (None, 1605)              162105    
_________________________________________________________________
dense_15 (Dense)             (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
____________________________________________

In [49]:
history = model.fit(padded_train_sequences, train_labels, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 9s 11ms/step - loss: 7.3528 - acc: 0.0190
Epoch 2/100
484/484 [==============================] - 5s 11ms/step - loss: 6.4393 - acc: 0.0222
Epoch 3/100
484/484 [==============================] - 6s 12ms/step - loss: 6.3949 - acc: 0.0255
Epoch 4/100
484/484 [==============================] - 5s 11ms/step - loss: 6.2698 - acc: 0.0273
Epoch 5/100
484/484 [==============================] - 6s 12ms/step - loss: 6.1422 - acc: 0.0354
Epoch 6/100
484/484 [==============================] - 6s 12ms/step - loss: 6.0803 - acc: 0.0393
Epoch 7/100
484/484 [==============================] - 6s 12ms/step - loss: 5.9879 - acc: 0.0422
Epoch 8/100
484/484 [==============================] - 6s 11ms/step - loss: 5.8987 - acc: 0.0486
Epoch 9/100
484/484 [==============================] - 6s 11ms/step - loss: 5.8107 - acc: 0.0506
Epoch 10/100
484/484 [==============================] - 6s 12ms/step - loss: 5.7025 - acc: 0.0542
Epoch 11/100
484/484 [=======

### 3. Dự đoán 10 từ tiếp theo

In [50]:
test_seq = 'despite of wrinkles'
# sinh ra 10 từ tiếp theo sau test_seq
# despite of wrinkles this thy golden time to heart's sight ' must '

num_words = 10
for i in range(num_words):
    token_list = pad_sequences([tokenizer.texts_to_sequences([test_seq])[0]], maxlen=max_sequence_len, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = tokenizer.index_word[predicted[0]]
    test_seq += ' ' + output_word
print(test_seq)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


despite of wrinkles this thy golden time or ever did place old lie
